In [2]:
import numpy as np
import glob
import os

In [3]:
import numpy as np
import cv2 as cv
import glob

# Criterios de terminación
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Preparar los puntos del objeto 3D, como (0,0,0), (1,0,0), (2,0,0) ..., (8,4,0)
# Se preparan para un tablero de ajedrez de 6x10, con esquinas internas de 5x9
objp = np.zeros((5*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:5].T.reshape(-1, 2)

# Arreglos para almacenar puntos de objeto y puntos de imagen de todas las imágenes
objpoints = []  # Puntos 3d en el espacio real
imgpoints = []  # Puntos 2d en el plano de imagen

In [4]:
# Lista de rutas a las imágenes
images = glob.glob('calib_images10x5/*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Encuentra las esquinas del tablero de ajedrez
    ret, corners = cv.findChessboardCorners(gray, (9,5), None)

    # Si se encuentran, añadir los puntos del objeto y de la imagen después de afinarlos
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Dibujar las esquinas encontradas en la imagen
        cv.drawChessboardCorners(img, (9,5), corners2, ret)
        
        # Construir la ruta del archivo de salida
        save_path = os.path.join('detected_borders', os.path.basename(fname))
        
        # Guardar la imagen con las esquinas dibujadas
        cv.imwrite(save_path, img)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

# Calibración de la cámara
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)



In [5]:
# Undistortion
# Toma una nueva imagen
img = cv.imread(r'calib_images2\2023_11_08_10_01_IMG_1098.JPG')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# 1. Usando cv.undistort()
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

# 2. Usando remapping
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult_remap.png', dst)

# Guardar los parámetros de la cámara para usarlos en el futuro
np.savez('calibration_data.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

In [6]:
# Carga los datos de calibración
with np.load('calibration_data.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

# Carga la imagen que quieres proyectar
overlay_image = cv.imread('test.jpeg')

# Captura de video desde la primera cámara conectada
cap = cv.VideoCapture(0)

# Criterios de terminación para la búsqueda de esquinas
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Dimensiones del tablero de ajedrez (por ejemplo, 6x10 para un 5x9 esquinas internas)
board_size = (9, 5)

while True:
    # Captura cuadro por cuadro
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Corrige la distorsión de la imagen
    frame_undistorted = cv.undistort(frame, mtx, dist, None, mtx)

    # Convierte a escala de grises
    gray = cv.cvtColor(frame_undistorted, cv.COLOR_BGR2GRAY)

    # Busca las esquinas del tablero de ajedrez
    ret, corners = cv.findChessboardCorners(gray, board_size, None)

    # Si se encuentran las esquinas, procede a superponer la imagen
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        
        # Obtén las esquinas de la imagen de superposición
        overlay_points = np.float32([[0, 0],
                                     [overlay_image.shape[1], 0],
                                     [overlay_image.shape[1], overlay_image.shape[0]],
                                     [0, overlay_image.shape[0]]])

        # Los puntos del tablero de ajedrez son los primeros y últimos puntos devueltos por findChessboardCorners
        chessboard_points = np.float32([corners2[0], corners2[8], corners2[-1], corners2[-9]])

        # Obtén la matriz de transformación
        matrix = cv.getPerspectiveTransform(overlay_points, chessboard_points)

        # Transforma la imagen de superposición
        warped_image = cv.warpPerspective(overlay_image, matrix, (frame_undistorted.shape[1], frame_undistorted.shape[0]))

        # Crea la máscara y su inversa a partir de la imagen de superposición transformada
        mask = np.zeros_like(gray)
        cv.fillConvexPoly(mask, np.int32(chessboard_points), 255)
        mask_inv = cv.bitwise_not(mask)

        # Superpone la imagen
        frame_undistorted = cv.bitwise_and(frame_undistorted, frame_undistorted, mask=mask_inv)
        frame_undistorted = cv.add(warped_image, frame_undistorted)

    # Muestra el cuadro resultante
    cv.imshow('Frame', frame_undistorted)

    # Presiona 'q' para salir
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cuando todo esté hecho, libera la captura
cap.release()
cv.destroyAllWindows()


KeyboardInterrupt: 

In [8]:
# Carga los datos de calibración
with np.load('calibration_data.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

# Carga la imagen que quieres proyectar
overlay_image = cv.imread('test.jpeg')

# Captura de video desde la primera cámara conectada
cap = cv.VideoCapture(0)

# Criterios de terminación para la búsqueda de esquinas
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Dimensiones del tablero de ajedrez (por ejemplo, 6x10 para un 5x9 esquinas internas)
board_size = (9, 5)

while True:
    # Captura cuadro por cuadro
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Corrige la distorsión de la imagen
    frame_undistorted = cv.undistort(frame, mtx, dist, None, mtx)

    # Convierte a escala de grises
    gray = cv.cvtColor(frame_undistorted, cv.COLOR_BGR2GRAY)

    # Ecualizar el histograma para mejorar el contraste
    gray = cv.equalizeHist(gray)

    # Aplica un suave desenfoque Gaussiano para reducir el ruido y mejorar la detección de esquinas
    gray = cv.GaussianBlur(gray, (5, 5), 0)

    # Busca las esquinas del tablero de ajedrez
    ret, corners = cv.findChessboardCorners(gray, board_size, None)

    # Si se encuentran las esquinas, procede a superponer la imagen
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        
        # Obtén las esquinas de la imagen de superposición
        overlay_points = np.float32([[0, 0],
                                     [overlay_image.shape[1], 0],
                                     [overlay_image.shape[1], overlay_image.shape[0]],
                                     [0, overlay_image.shape[0]]])

        # Los puntos del tablero de ajedrez son los primeros y últimos puntos devueltos por findChessboardCorners
        chessboard_points = np.float32([corners2[0], corners2[8], corners2[-1], corners2[-9]])

        # Obtén la matriz de transformación
        matrix = cv.getPerspectiveTransform(overlay_points, chessboard_points)

        # Transforma la imagen de superposición
        warped_image = cv.warpPerspective(overlay_image, matrix, (frame_undistorted.shape[1], frame_undistorted.shape[0]))

        # Crea la máscara y su inversa a partir de la imagen de superposición transformada
        mask = np.zeros_like(gray)
        cv.fillConvexPoly(mask, np.int32(chessboard_points), 255)
        mask_inv = cv.bitwise_not(mask)

        # Superpone la imagen
        frame_undistorted = cv.bitwise_and(frame_undistorted, frame_undistorted, mask=mask_inv)
        frame_undistorted = cv.add(warped_image, frame_undistorted)

    # Muestra el cuadro resultante
    cv.imshow('Frame', frame_undistorted)

    # Presiona 'q' para salir
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cuando todo esté hecho, libera la captura
cap.release()
cv.destroyAllWindows()


KeyboardInterrupt: 